In [29]:
import pandas as pd
import csv 
import geopandas as gpd
from datetime import timedelta
from shapely.geometry import Point
from shapely import wkt

In [12]:
C_im_date = pd.read_csv('../data/Cienega/CienegaImageryDates.csv', parse_dates=['date'])
C_sur_date = pd.read_csv('../data/Cienega/Cienega_survey_dates.csv', parse_dates=['Cienega date'])
C_sur_date['Cienega date'] = pd.to_datetime(C_sur_date['Cienega date'])
C_im_date['date'] = pd.to_datetime(C_im_date['date'])

C_hyd = pd.read_csv('../data/Cienega/CienegaHydroData.csv')
C_hyd['datetime'] = pd.to_datetime(C_hyd['datetime'])

C_sur_date = C_sur_date.dropna(subset=['Cienega date'])
C_im_date = C_im_date.dropna(subset=['date'])

C_surveyData = pd.read_csv('../data/Cienega/Cienega_surveyData.csv')

In [13]:
matching_dates = []
tolerance = timedelta(days = 5)


for date1 in C_sur_date['Cienega date']:
    exact_date = False
    tol = False 
    for date2 in C_im_date['date']:
        if date1 == date2:
            matching_dates.append({'Survey': date1, 'Imagery': date2})
            exact_date = True
    if not exact_date:
        for date2 in C_im_date['date']:
            if abs(date1 - date2) <= tolerance:
                matching_dates.append({'Survey': date1, 'Imagery': date2})
                tol = True
        if not tol:
            for date2 in C_im_date['date']:
                if abs(date1-date2) < timedelta(days = 10): 
                    matching_dates.append({'Survey': date1, 'Imagery': date2})


matching_dates_df = pd.DataFrame(matching_dates)



In [14]:
C_datessurData = pd.merge(matching_dates_df, C_hyd, left_on = 'Survey', right_on = 'datetime', how = 'left')
C_datesimData = pd.merge(matching_dates_df, C_hyd, left_on = 'Imagery', right_on = 'datetime')
C_datessurData = C_datessurData.drop(columns = ['Imagery','datetime'])
C_datesimData = C_datesimData.drop(columns = ['Survey','datetime'])

In [15]:
def sum_pdatesbetween(d1, d2):
    r = pd.date_range(start=min(d1,d2), end=max(d1,d2))
    return C_hyd[C_hyd['datetime'].isin(r)]['P [mm]'].sum()

In [16]:
Ch = pd.DataFrame([])

Ch['Survey'] = matching_dates_df['Survey']
Ch['Imagery'] = matching_dates_df['Imagery']
Ch['sum_P'] = [sum_pdatesbetween(C_datessurData.loc[i, 'Survey'], C_datesimData.loc[i, 'Imagery']) for i in range(len(Ch))]
Ch['Q_diff [%]'] = (C_datessurData['Q [mm/d]'] - C_datesimData['Q [mm/d]']) / C_datessurData['Q [mm/d]'] * 100
Ch['Use/not'] = ['use', 'use', 'use', 'use', 'not', 'not', 'use?',
                 'not', 'only option', 'not', 'not', 'use', 'not',
                 'not', 'not', 'use', 'not', 'not', 'not', 'not',
                 'use', 'use', 'use', 'only option','use', 'use', 
                 'use','use', 'not', 'not', 'not', 'not?', 'use', 
                 'not', 'use', 'use', 'use', 'use', 'use', 'not', 
                 'not', 'not', 'not', 'use', 'not', 'not', 'not', 
                 'not', 'not', 'use', 'not', 'not', 'not', 'use', 
                 'not', 'use', 'use']
Ch

,Survey,Imagery,sum_P,Q_diff [%],Use/not
0,2016-09-23,2016-09-22,0.000,0.000000,use
1,2017-03-16,2017-03-13,0.000,0.000000,use
2,2017-06-09,2017-06-09,0.000,0.000000,use
3,2017-09-19,2017-09-21,0.000,4.587156,use
4,2017-09-19,2017-09-23,0.000,5.504587,not
5,2017-12-08,2017-12-04,0.000,-21.495327,not
6,2017-12-08,2017-12-07,0.000,-2.803738,use?
7,2017-12-08,2017-12-09,0.000,8.411215,not
8,2018-03-23,2018-03-17,0.000,20.754717,only option
9,2018-03-23,2018-03-31,0.000,-26.415094,not


In [17]:
Ch.to_csv('../data/Cienega/Cienega_survey_imagery_HydroData.csv', encoding='utf-8', index=False)

In [30]:
C_surveyData['geometry'] = C_surveyData['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(C_surveyData, geometry='geometry', crs='EPSG:26912')

gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y
gdf = gdf[['geometry', 'x', 'y', 'Year', 'wetdry']]
gdf

,geometry,x,y,Year,wetdry
0,POINT (533719.252 3542427.373),533719.252411,3.542427e+06,6-25-2004,dry
1,POINT (533719.787 3542422.402),533719.786671,3.542422e+06,6-25-2004,dry
2,POINT (533720.321 3542417.431),533720.320931,3.542417e+06,6-25-2004,dry
3,POINT (533720.855 3542412.459),533720.855191,3.542412e+06,6-25-2004,dry
4,POINT (533721.389 3542407.488),533721.389451,3.542407e+06,6-25-2004,dry
...,...,...,...,...,...
237685,POINT (533717.227 3542446.217),533717.227336,3.542446e+06,9-5-2013,dry
237686,POINT (533717.762 3542441.246),533717.761591,3.542441e+06,9-5-2013,dry
237687,POINT (533718.296 3542436.274),533718.295846,3.542436e+06,9-5-2013,dry
237688,POINT (533718.830 3542431.303),533718.830102,3.542431e+06,9-5-2013,dry
